In [1]:
%reset -sf

In [2]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
# d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [18]:
srr = """
""".strip()

# with open("/Users/hkmac/Downloads/input.txt") as f:
#     srr = f.read().strip()

# brr = srr.split("\n\n")
# brr = [sum(int(x) for x in srr.split("\n")) for srr in brr]

brr = srr.split("\n")

In [19]:
sum(sorted(brr)[-3:])

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
def test(arr):
    res = 0
    return res